**to prevent disconnection of virtual machine paste this to console (Ctrl+Shift+I) and hit enter :**
function ClickConnect(){
console.log("Working");
document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
}
var clicker=setInterval(ClickConnect,60000)<p>
**to stop:**clearInterval(clicker);

In [ ]:
#@title Initialize environment { vertical-output: true, display-mode: "form" }
import ee
from google.colab import drive
ee.Authenticate()
ee.Initialize()
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)
drive.mount('/content/gdrive', force_remount=True)
import os
from google.colab import files
from google.colab import output
import time
import numpy as np
from datetime import datetime
from scipy.interpolate import interp1d
import glob
output.clear()
!pip install rasterio
import rasterio
from rasterio.windows import Window
from rasterio import merge
from rasterio.fill import fillnodata
from rasterio import features
from rasterio.windows import Window
from rasterio.transform import Affine
output.clear()
!pip install fiona
output.clear()
import fiona
import pickle
from distutils.dir_util import copy_tree
if not os.path.isdir('keras_unet_collection'):
  copy_tree('gdrive/MyDrive/Arabian/keras_unet_collection', 'keras_unet_collection')
output.clear()
from keras_unet_collection.models import transunet_2d
from keras_unet_collection.losses import *
from keras_unet_collection.transformer_layers import patch_extract, patch_embedding
from keras_unet_collection.activations import GELU, Snake
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import custom_object_scope
import tensorflow as tf
import random
import gc
import shutil

In [ ]:
#@title Prepare dataset on EE side+load functions { vertical-output: true, display-mode: "form" }
# generate AOI from ".gpkg"
bands=28
size=512
halfsize=size//2
overlap=halfsize//2
parent='gdrive/MyDrive/GEE_Beholder_Tiles/'
shapefile=parent+'val_grid1_4326.gpkg'
shapefile3857=parent+'val_grid1_3857.gpkg'
move_path=parent+'moved/'
predicted_path=parent+'result/'
std_path=parent+'std/'

polygon_features = []
with fiona.open(shapefile) as layer:
    for feature in layer:
        polygon_features.append(ee.Feature(feature))
AOI = ee.FeatureCollection(polygon_features)#.union().geometry().bounds()
print(len(polygon_features),'polygons')
#obtain masked image collection on basis of AOI and parameters below
STARTYEAR = 2019
ENDYEAR = 2020
STARTDAY=70
ENDDAY=310
INTERVAL = ENDDAY - STARTDAY
scale = 30
my_bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12',
                'NDVI', 'ARVI', 'GNDVI', 'CIG', 'FE_2', 'FE_3', 'FO', 'FS', 'FIA', 'CCA',
                'NDSI', 'FI', 'FI_1', 'FI_2',
                'NDSnowI', 'WM', 'NBR', 'NBR_2' ];
CLOUD_FILTER = 70
CLD_PRB_THRESH = 10
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 80

NODATA_PIXEL_PERCENTAGE = 80

#RELIEF
dem = ee.Image("USGS/SRTMGL1_003");
slope = ee.Terrain.slope(dem).clip(AOI)
aspect = ee.Terrain.aspect(dem).clip(AOI)
aspect_modifed = aspect.subtract(180).abs()

WorldCover_ds = ee.ImageCollection("ESA/WorldCover/v100").first();
settlements_mask = ee.FeatureCollection('projects/gee-antonbiatov/assets/beholder_arabian_peninsula/settlements_mask')

def to_int16(image):
  return image.int16()
def addNDVI(image):# add NDVI band   (NIR - RED) / (NIR + RED)
  return image.addBands(image.expression('int16(((b("B8") - b("B4")) / (b("B8") + b("B4"))+1)*1000)').rename('NDVI'))
def addARVI(image):# add ARVI band    (NIR – (2 * RED) + BLUE) / (NIR + (2 * RED) + BLUE)
  return image.addBands(image.expression('int16( ( (b("B8") - (2 * b("B4")) + b("B2")) / ( b("B8") + (2 * b("B4")) + b("B2") ) +1 ) * 1000)').rename('ARVI'))
def addGNDVI(image): # add GNDVI band    (NIR - GREEN) / (NIR + GREEN)
  return image.addBands(image.expression('int16(((b("B8") - b("B3")) / (b("B8") + b("B3"))+1)*1000)').rename('GNDVI'))
def addCIG(image):# add CIG band (Chlorophyll Index Green)   NIR / GREEN
  return image.addBands(image.expression('int16((b("B8") / b("B3"))*1000)').rename('CIG'))
def addFE_2(image):# add FE_2 band (Ferric Iron, Fe2+)   SWIR2 / NIR + GREEN / RED
  return image.addBands(image.expression('int16(((b("B12") / b("B8")) + (b("B3") / b("B4")))*1000)').rename('FE_2'))
def addFE_3(image): # add FE_3 band (Ferric Iron, Fe3+)   GREEN / RED
  return image.addBands(image.expression('int16((b("B3") / b("B4"))*1000)').rename('FE_3'))
def addFO(image): # add FO band (Ferric Oxides)   SWIR1 / NIR
  return image.addBands(image.expression('int16((b("B11") / b("B8"))*1000)').rename('FO'))
def addFS(image):# add FS band (Ferrous Silicates (Ferric Iron Alteration) )   SWIR2 / SWIR1
  return image.addBands(image.expression('int16((b("B12") / b("B11"))*1000)').rename('FS'))
def addFIA(image):# add FIA band ( Ferrous Iron Alteration )   SWIR1 / RedEdge
  return image.addBands(image.expression('int16((b("B11") / b("B5"))*1000)').rename('FIA'))
def addCCA(image):# add CCA band ( Clay Carbonate Alteration; Clay-sulfate-mica-marble index )   SWIR1 / SWIR2
  return image.addBands(image.expression('int16((b("B11") / b("B12"))*1000)').rename('CCA'))
def addNDSI(image): # add NDSI band ( Normalized Different Senescence Index )   (SWIR1 - RED) / (SWIR1 + RED)
  return image.addBands(image.expression('int16(((b("B11") - b("B4")) / (b("B11") + b("B4"))+1)*1000)').rename('NDSI'))
def addFI(image):# add FI band ( Ferrous iron – coarse-grained ferric iron – fire ash index )   (GREEN  + SWIR1 ) / (RED + NIR)
  return image.addBands(image.expression('int16(((b("B3") + b("B11")) / (b("B4") + b("B8"))+1)*1000)').rename('FI'))
def addFI_1(image):# add FI_1 band ( Ferric iron 1 “redness” index )   RED / BLUE
  return image.addBands(image.expression('int16((b("B4") / b("B2"))*1000)').rename('FI_1'))
def addFI_2(image): # add FI_2 band ( Ferric iron index 2 )   (RED / BLUE) * ((RED + SWIR1 ) / NIR)
  return image.addBands(image.expression('int16((b("B4") + b("B2")) * ((b("B4") + b("B11")) / b("B8")))').rename('FI_2'))
def addNDSnowI(image): # add NDSnowI band ( Normalized Difference Snow Index )   (GREEN – SWIR1 ) / (GREEN + SWIR1 )
  return image.addBands(image.expression('int16(((b("B3") - b("B11")) / (b("B3") + b("B11"))+1)*1000)').rename('NDSnowI'))
def addWM(image):# add WM band ( Water Masking )   BLUE / SWIR1
  return image.addBands(image.expression('int16((b("B2") / b("B11"))*1000)').rename('WM'))
def addNBR(image):# add NBR band ( Normalized Burn Ratio )   (NIR - SWIR2) / (NIR + SWIR2)
  return image.addBands(image.expression('int16(((b("B8") - b("B12")) / (b("B8") + b("B12"))+1)*1000)').rename('NBR'))
def addNBR_2(image): # add NBR_2 band ( Normalized Burn Ratio 2 )   (SWIR1 – SWIR2) / (SWIR1 + SWIR2)
  return image.addBands(image.expression('int16(((b("B11") - b("B12")) / (b("B11") + b("B12"))+1)*1000)').rename('NBR_2'))

#  ////////////////////////////////////////////////////////////////////////////////
# Function to calculate illumination condition (IC). Function by Patrick Burns and Matt Macander
def illuminationCondition(img):

    # Extract image metadata about solar position
    SZ_rad = ee.Image.constant(ee.Number(img.get('MEAN_SOLAR_ZENITH_ANGLE'))).multiply(3.14159265359).divide(180).clip(img.geometry().buffer(10000))
    SA_rad = ee.Image.constant(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).multiply(3.14159265359).divide(180)).clip(img.geometry().buffer(10000))
    # Creat terrain layers
    slp = ee.Terrain.slope(dem).clip(img.geometry().buffer(10000))
    slp_rad = ee.Terrain.slope(dem).multiply(3.14159265359).divide(180).clip(img.geometry().buffer(10000))
    asp_rad = ee.Terrain.aspect(dem).multiply(3.14159265359).divide(180).clip(img.geometry().buffer(10000))

    # Calculate the Illumination Condition (IC)
    # slope part of the illumination condition
    cosZ = SZ_rad.cos()
    cosS = slp_rad.cos()
    slope_illumination = cosS.expression("cosZ * cosS",
                                          {'cosZ': cosZ,
                                           'cosS': cosS.select('slope')})
    # aspect part of the illumination condition
    sinZ = SZ_rad.sin()
    sinS = slp_rad.sin()
    cosAziDiff = (SA_rad.subtract(asp_rad)).cos()
    aspect_illumination = sinZ.expression("sinZ * sinS * cosAziDiff",
                                           {'sinZ': sinZ,
                                            'sinS': sinS,
                                            'cosAziDiff': cosAziDiff})
    # full illumination condition (IC)
    ic = slope_illumination.add(aspect_illumination)

    # Add IC to original image
    img_plus_ic = ee.Image(img.addBands(ic.rename('IC')).addBands(cosZ.rename('cosZ')).addBands(cosS.rename('cosS')).addBands(slp.rename('slope')))
    return img_plus_ic

  # //////////////////////////////////////////////////////////////////////////////
  # Function to apply the Sun-Canopy-Sensor + C (SCSc) correction method to each
  # image. Function by Patrick Burns and Matt Macander
def illuminationCorrection(img):
    props = img.toDictionary()
    st = img.get('system:time_start')

    img_plus_ic = img;
    mask1 = img_plus_ic.select('B8').gt(-0.1)
    mask2 = (img_plus_ic.select('slope').gte(5)
                            .And(img_plus_ic.select('IC').gte(0))
                            .And(img_plus_ic.select('B8').gt(-0.1)))
    img_plus_ic_mask2 = ee.Image(img_plus_ic.updateMask(mask2))

    # Specify Bands to topographically correct
    # bandList = ['B2','B3','B4','B8','B11','B12']
    # bandList = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    # bandList = [ 'B7'] # возникает ошибка :
    # Error: Error in map(ID=20200825T180919_20200825T182151_T12SWJ):
    # Number.divide: Parameter 'left' is required. (Error code: 3)
    bandList = ['B2', 'B3', 'B4', 'B5', 'B6', 'B8', 'B8A', 'B11', 'B12']
    compositeBands = img.bandNames()
    nonCorrectBands = img.select(compositeBands.removeAll(bandList))

    geom = ee.Geometry(img.get('system:footprint')).bounds().buffer(10000)

    def apply_SCSccorr(band):
        method = 'SCSc';
        out = (img_plus_ic_mask2.select('IC', band)
                            .reduceRegion(reducer= ee.Reducer.linearFit(), # Compute coefficients: a(slope), b(offset), c(b/a)
              #                           geometry= ee.Geometry(img.geometry().buffer(-5000)), # trim off the outer edges of the image for linear relationship
                                         geometry= ee.Geometry(img.geometry()),
                                         scale= 30,
                                         maxPixels= 10000000000
                                         ))

#        if (out == 'null'):
#            return img_plus_ic_mask2.select(band)
#        else:
#            out_a = ee.Number(out.get('scale'))
#            out_b = ee.Number(out.get('offset'))
#            out_c = out_b.divide(out_a)

        out_a = ee.Number(out.get('scale'))
        out_b = ee.Number(out.get('offset'))
        out_c = out_b.divide(out_a)
      # Apply the SCSc correction
        SCSc_output = img_plus_ic_mask2.expression(
        "((image * (cosB * cosZ + cvalue)) / (ic + cvalue))", {
        'image': img_plus_ic_mask2.select(band),
        'ic': img_plus_ic_mask2.select('IC'),
        'cosB': img_plus_ic_mask2.select('cosS'),
        'cosZ': img_plus_ic_mask2.select('cosZ'),
        'cvalue': out_c
        })

        return SCSc_output.int16()


    img_SCSccorr = ee.Image(list(map(apply_SCSccorr,bandList))).addBands(img_plus_ic.select('IC'))

    bandList_IC = ee.List([bandList, 'IC']).flatten()
    img_SCSccorr = img_SCSccorr.unmask(img_plus_ic.select(bandList_IC)).select(bandList).int16()

    return img_SCSccorr.addBands(nonCorrectBands).set(props).set('system:time_start',st)
#    return img_SCSccorr.addBands(nonCorrectBands).setMulti(props).set('system:time_start',st)

def topoCorrection(collection):
    #call function to calculate illumination condition
    collection = collection.map(illuminationCondition)
    #call function to calculate illumination Correction
    collection = collection.map(illuminationCorrection)
    return(collection)

def maskVegetation(image):
  qa_1 = image.select('ARVI')
  mask = qa_1.lt(1500)#.And(qa_1.gt(1000));
  return image.updateMask(mask)
def maskCloud2(image):
  qa_1 = image.select('B2')
  qa_2 = image.select('B3')
  mask = qa_1.lt(2700).And(qa_2.lt(2700))
  return image.updateMask(mask)

def get_s2_sr_cld_col(little_aoi, STARTYEAR, ENDYEAR,STARTDAY,ENDDAY):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(little_aoi)
        .filter(ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'))
        .filter(ee.Filter.dayOfYear(STARTDAY, ENDDAY))
        .filter(ee.Filter.lte('NODATA_PIXEL_PERCENTAGE', NODATA_PIXEL_PERCENTAGE))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        .map(addNDVI).map(addARVI).map(addGNDVI).map(addCIG).map(addFE_2).map(addFE_3).map(addFO).map(addFS).map(addFIA).map(addCCA)
        .map(addNDSI).map(addFI).map(addFI_1).map(addFI_2).map(addNDSnowI).map(addWM).map(addNBR).map(addNBR_2)
        #.map(maskVegetation)
        #.map(maskCloud2)
        .map(to_int16))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(little_aoi)
        .filter(ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'))
        .filter(ee.Filter.dayOfYear(STARTDAY, ENDDAY)))
    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{'primary': s2_sr_col,'secondary': s2_cloudless_col,
    'condition': ee.Filter.equals(**{'leftField': 'system:index','rightField': 'system:index'})}))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)
    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).rename('dark_pixels').multiply(not_water)
    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)
    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)
    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()
    # Subset reflectance bands and update their masks, return the result.
    return img.select(my_bands).updateMask(not_cld_shdw)

# vector invert mask. Need to have field 'val' = 1
def vect_invert_mask(vect):
    image_reduced = vect.reduceToImage(['val'], 'mean')
    pre_mask = image_reduced.unmask(0)
    invert_pre_mask = pre_mask.Not()
#    invert_mask = invert_pre_mask.selfMask()
    return invert_pre_mask

# Define a kernel for water and landscape bufferization.
# kernel = ee.Kernel.circle(**{'radius': 1});
kernel = ee.Kernel.circle(**{'radius': 2});

# Mask generator
# 10 Trees, 20 Shrubland, 30 Grassland, 40 Cropland, 50 Built-up, 60 Barren / sparse vegetation,
# 70 Snow and ice, 80 Open water, 90 Herbaceous wetland, 95 Mangroves, 100 Moss and lichen

landscape_mask = (WorldCover_ds.select('Map').eq(30)  # Grassland
    .Or(WorldCover_ds.select('Map').eq(40))  # Cropland
    .Or(WorldCover_ds.select('Map').eq(60))  # Barren / sparse vegetation
    .Or(WorldCover_ds.select('Map').eq(100)) # Moss and lichen
    .rename('mask'))

landscape_mask_buffered = landscape_mask.focalMin(**{'kernel': kernel, 'iterations': 1});
mask = landscape_mask_buffered.updateMask(vect_invert_mask(settlements_mask)).selfMask().rename('mask')
#def create_mask_tile(vector):
#    mask_tile = mask.clip(vector).int16()
#    return mask_tile

def export_tile(poly,id):
  AOI=ee.Geometry(poly.geometry())
  s2_sr_cld_col = get_s2_sr_cld_col(AOI, STARTYEAR, ENDYEAR,STARTDAY,ENDDAY)
  #'''
  s2_sr_cld_col_corr=topoCorrection(s2_sr_cld_col)
  s2_sr_msk_col = (s2_sr_cld_col_corr.map(add_cld_shdw_mask).map(apply_cld_shdw_mask))
  '''
  s2_sr_msk_col = (s2_sr_cld_col.map(add_cld_shdw_mask).map(apply_cld_shdw_mask))
  '''
  med=s2_sr_msk_col.median().clip(AOI)
  med_msk = med.updateMask(mask)
  final_dataset = med_msk.addBands(dem.rename('elevation')).addBands(slope.multiply(10).rename('slope')).addBands(aspect_modifed.multiply(10).rename('aspect')).clip(AOI).int16()
  #merged_masked_1 = merged.updateMask(vect_invert_mask(settlements_roads))
  #merged_masked = merged_masked_1.updateMask(vect_invert_mask(wells_mask_buffer))
  #bandNameslist=final_dataset.bandNames().getInfo()
  #bandNameslist=list(map(lambda bandname:bandname[:8],bandNameslist[::10]))
  #id=poly.getInfo()["id"]
  print('exporting to gdrive/MyDrive/GEE_Beholder_Tiles/'+id+'.tif...')
  #print(bandNameslist)
  #print(len(bandNameslist),'dates,',len(np.unique(bandNameslist)),'unique')
  np.save('gdrive/MyDrive/GEE_Beholder_Tiles/'+id+'.npy',s2_sr_cld_col)
  task=ee.batch.Export.image.toDrive(image = final_dataset, region=AOI, folder="GEE_Beholder_Tiles",
    description = id, crs='EPSG:3857', scale = scale, maxPixels = 1e10,fileFormat = 'GeoTIFF')
  task.start()
  return task

if not os.path.isdir(move_path):os.mkdir(move_path)
if not os.path.isdir(predicted_path):os.mkdir(predicted_path)
if not os.path.isdir(std_path):os.mkdir(std_path)

custom_objects = {"patch_extract": patch_extract,"patch_embedding":patch_embedding,"GELU":GELU,"focal_tversky":focal_tversky}
models=[]
with custom_object_scope(custom_objects):
  for f in glob.glob(parent+'models/*.hdf5'):
    models.append(load_model(f))

percentile=np.load(parent+'percentile.npy').astype(np.float32)

def interpolate(file):
  masked_bands=28
  other_bands=3
  src=rasterio.open(file)
  profile = src.profile
  profile.update({"count":masked_bands+other_bands+1})
  dest= rasterio.open(file +'f', 'w', **profile)
  arr=src.read(1)
  mask=~(arr==0)
  dest.write_band(1, fillnodata(arr, mask=mask, max_search_distance=10, smoothing_iterations=0)-1)
  for band in range(2,masked_bands+1):
      arr=src.read(band)
      dest.write_band(band, fillnodata(arr, mask=mask, max_search_distance=10, smoothing_iterations=0)-1)
  for band in range(masked_bands+1,masked_bands+1+other_bands):
    dest.write_band(band, src.read(band))
  dest.write_band(masked_bands+other_bands+1, mask)
  src.close()
  dest.close()
  shutil.move(file +'f',file)

def infer(tiles,tile_index,direct):
        X,out_transform = rasterio.merge.merge(tiles)
        x,y=np.abs(direct)
        X=X[:31,halfsize*y:halfsize*(y+2),halfsize*(x):halfsize*(x+2)]

        X[X<-1]=-1
        X=X.astype(np.float32)
        X=np.transpose(X,(1,2,0))/percentile
        X[X>1.0]=1.0

        Y=models[0](np.expand_dims(X,0)[:,:,:,:bands])[:,overlap:size-overlap,overlap:size-overlap,:]
        for model in models[1:]:
          Y=np.concatenate((Y,model(np.expand_dims(X,0)[:,:,:,:bands])[:,overlap:size-overlap,overlap:size-overlap,:]),axis=3)
        std=np.std(Y,axis=3)
        Y=np.mean(Y,axis=3)

        out_meta = tiles[0].meta.copy()
        out_transform = Affine.translation(*rasterio.transform.xy(out_transform,halfsize*y+overlap,halfsize*x+overlap,offset='ul'))
        out_transform *= Affine.scale(30, -30)
        out_meta.update({"height": Y.shape[1],"width": Y.shape[2],"transform": out_transform,"count":1,"dtype":np.float32})

        with rasterio.open(predicted_path+str(tuple(tile_index*2+1+direct))[1:-1].replace(', ','_')+'.tif', "w", **out_meta) as dest1:
            dest1.write(Y)
        with rasterio.open(std_path+str(tuple(tile_index*2+1+direct))[1:-1].replace(', ','_')+'.tif', "w", **out_meta) as dest2:
            dest2.write(std)
        Index1[tuple(tile_index*2+1+direct)] = str(tuple(tile_index*2+1+direct))
        np.save(parent+'Index1.npy',Index1)

def get_dims_coord(bounds):
    return np.array([bounds[2]-bounds[0], bounds[3]-bounds[1]])

def register_tile(file):
    print('registering',file,'...')
    tile = rasterio.open(file)
    tile_index = np.around((np.array(rasterio.transform.xy(tile.transform,0,0))-start_coord)/tile_dims_coord,0).astype(np.uint16)+[1,0]
    direct = np.array([0,0])
    infer([tile],tile_index,direct)

    vflag=0
    for vdirect in np.array([[0,-1],[0,1]]):
        v = Index[tuple(tile_index+vdirect)]
        if v != "":
            vtile=rasterio.open(v)
            del v
            infer([tile,vtile],tile_index,vdirect)
            for hdirect in np.array([[-1,0],[1,0]]):
                h = Index[tuple(tile_index+hdirect)]
                if h != "":
                    htile=rasterio.open(h)
                    del h
                    if Index1[tuple(tile_index*2+1+hdirect)]=="":
                        infer([tile,htile],tile_index,hdirect)
                    cdirect = vdirect + hdirect
                    c = Index[tuple(tile_index+cdirect)]
                    if c != "":
                        ctile=rasterio.open(c)
                        del c
                        infer([tile,vtile,htile,ctile],tile_index,cdirect)
                        ctile.close()
                    htile.close()
            vtile.close()

            vflag=1
    if vflag==0:
        for hdirect in np.array([[-1,0],[1,0]]):
            h = Index[tuple(tile_index+hdirect)]
            if h != "":
                htile=rasterio.open(h)
                del h
                infer([tile,htile],tile_index,hdirect)
                htile.close()
    tile.close()

    Index[tuple(tile_index)] = file
    np.save(parent+'Index.npy',Index)
    #print(np.flip(Index.transpose(),0))

    directs0=np.array([[i,ii] for i in range(-1,2) for ii in range(-1,2)])
    directs=directs0[np.any(directs0!=0,axis=1)]
    for d in directs:
        if Index[tuple(tile_index+d)] != "":
            Counter[tuple(tile_index+d)] += 1
            Counter[tuple(tile_index)] += 1
            np.save(parent+'Counter.npy',Counter)
    for d in directs0:
        if Counter[tuple(tile_index+d)]==8:
            file=Index[tuple(tile_index+d)]
            shutil.move(file, move_path+file.split('/')[-1])

if not os.path.isfile(parent+'coords.npy'):
    with fiona.open(shapefile3857) as source:
        tile_dims_coord = get_dims_coord(features.bounds(next(iter(source))['geometry']))
        dims = np.around(get_dims_coord(source.bounds)/tile_dims_coord,0).astype(np.uint16)
        Index = np.array([["" for _ in range(dims[1]+2)] for _ in range(dims[0]+2)],dtype = object)
        Counter = np.zeros(dims+2,dtype = np.uint8)
        Index1 = np.array([["" for _ in range(Index.shape[1]*2+1)] for _ in range(Index.shape[0]*2+1)],dtype=object)
        start_coord = np.array(source.bounds)[:2]
    np.save(parent+'coords.npy',np.array([tile_dims_coord,start_coord]))
    np.save(parent+'Index.npy',Index)
    np.save(parent+'Counter.npy',Counter)
    np.save(parent+'Index1.npy',Index1)
else:
    tile_dims_coord,start_coord = np.load(parent+'coords.npy')[[0,1]]
    Index = np.load(parent+'Index.npy',allow_pickle=True)
    Index1 = np.load(parent+'Index1.npy',allow_pickle=True)
    Counter = np.load(parent+'Counter.npy',allow_pickle=True)

60 polygons


In [ ]:
#@title Export+preprocessing of geodata { vertical-output: true, display-mode: "form" }
first =  1#@param {type:"integer"}
last =  60#@param {type:"integer"}
#automatic batch export from Earth Engine and postprocessing
#from pprint import pprint

if os.path.isfile(parent+'tasks.pkl'):
  tasks=pickle.load(open(parent+'tasks.pkl','rb'))
  names=list(np.load(parent+'names.npy'))
else:
  tasks=[]
  names=[]

for id,poly in enumerate(polygon_features[first-1:last]):
  name=parent+str(id+first)#poly.getInfo()["id"]
  if name+'.tif' not in Index:
      if not os.path.isfile(name+'.npy'):
        names.append([name+'.tif',80])
        tasks.append(export_tile(poly,str(id+first)))
        pickle.dump(tasks, open(parent+'tasks.pkl', 'wb'))
        np.save(parent+'names.npy',names)
        #time.sleep(20)
      i=0
      while i<len(names):
        if tasks[i].status()['state']=='FAILED':
          #time.sleep(10)
          names[i][1] = int(names[i][1]) - 10
          print(names[i][1])
          NODATA_PIXEL_PERCENTAGE = int(names[i][1])
          tile_i=int(names[i][0].split('/')[-1][:-4])-1
          tasks[i] = export_tile(polygon_features[tile_i],str(tile_i+1))
          pickle.dump(tasks, open(parent+'tasks.pkl', 'wb'))
          NODATA_PIXEL_PERCENTAGE = 80
        if tasks[i].status()['state']=='RUNNING':
          if int(tasks[i].status()['attempt'])>2:
            #time.sleep(10)
            names[i][1] = int(names[i][1]) - 10
            NODATA_PIXEL_PERCENTAGE = int(names[i][1])
            tile_i=int(names[i][0].split('/')[-1][:-4])-1
            tasks[i].cancel()
            tasks[i] = export_tile(polygon_features[tile_i],str(tile_i+1))
            pickle.dump(tasks, open(parent+'tasks.pkl', 'wb'))
            NODATA_PIXEL_PERCENTAGE = 80
        if tasks[i].status()['state']=='COMPLETED':
          time.sleep(5)
          while not os.path.isfile(names[i][0]):
            time.sleep(5)
          interpolate(names[i][0])
          time.sleep(20)
          register_tile(names[i][0])
          names.pop(i)
          tasks.pop(i)
          pickle.dump(tasks, open(parent+'tasks.pkl', 'wb'))
          np.save(parent+'names.npy',names)
        else:
          i+=1
        if (len(names)>150) and (i==len(names)):i=0

while len(names):
  i=0
  while i<len(names):
    if tasks[i].status()['state']=='FAILED':
      names[i][1] = int(names[i][1]) - 10
      NODATA_PIXEL_PERCENTAGE = int(names[i][1])
      tile_i=int(names[i][0].split('/')[-1][:-4])-1
      tasks[i] = export_tile(polygon_features[tile_i],str(tile_i+1))
      pickle.dump(tasks, open(parent+'tasks.pkl', 'wb'))
      NODATA_PIXEL_PERCENTAGE = 80
    if tasks[i].status()['state']=='RUNNING':
      if int(tasks[i].status()['attempt'])>2:
        names[i][1] = int(names[i][1]) - 10
        NODATA_PIXEL_PERCENTAGE = int(names[i][1])
        tile_i=int(names[i][0].split('/')[-1][:-4])-1
        tasks[i].cancel()
        tasks[i] = export_tile(polygon_features[tile_i],str(tile_i+1))
        pickle.dump(tasks, open(parent+'tasks.pkl', 'wb'))
        NODATA_PIXEL_PERCENTAGE = 80
    if tasks[i].status()['state']=='COMPLETED':
      time.sleep(15)
      while not os.path.isfile(names[i][0]):
        time.sleep(10)
      interpolate(names[i][0])
      time.sleep(20)
      register_tile(names[i][0])
      names.pop(i)
      tasks.pop(i)
      pickle.dump(tasks, open(parent+'tasks.pkl', 'wb'))
      np.save(parent+'names.npy',names)
    else:
      i+=1

In [ ]:
names[1][1]=90

In [ ]:
len(names)

98

In [ ]:
tasks=[]
NODATA_PIXEL_PERCENTAGE = 40
tasks.append(export_tile(polygon_features[0],'1'))
tasks.append(export_tile(polygon_features[1],'2'))

exporting to gdrive/MyDrive/GEE_Beholder_Tiles/1.tif...
exporting to gdrive/MyDrive/GEE_Beholder_Tiles/2.tif...


In [ ]:
print(tasks[1].status()['state'])

In [ ]:
pickle.dump(tasks, open('1.pkl', 'wb'))

In [ ]:
a=pickle.load(open('1.pkl','rb'))

In [ ]:
print(a[0].status())
print(a[1].status())

In [ ]:
a[0].cancel()

In [ ]:
x=[]

In [ ]:
x.append(['b',2])

In [ ]:
x[0][1]

In [ ]:
for task in tasks:task.cancel()

In [ ]:
#only export
N=3
for i in range(1,2):
  export_tile(polygon_features[i-1],str(i))
  time.sleep(10)